In [1]:
import os
%pwd

'c:\\Users\\naura\\Desktop\\end-to-end-ML\\computer-vision\\Bone-fracture-xray-classification\\notebooks'

In [2]:
os.chdir("../")

In [3]:
from pydantic import BaseModel
from pathlib import Path

class BaseModelConfig(BaseModel):
    root_dir: Path
    base_model_path: Path
    custom_trained_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [4]:
from boneFractureClassification.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from boneFractureClassification.utils.common import read_yaml, create_directories

In [5]:
class ModelConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_base_model_config(self) -> BaseModelConfig:
        
        config = self.config.base_model
        
        create_directories([config.root_dir])
        create_directories([config.custom_train_model_dir])
        
        base_model_config = BaseModelConfig(
            root_dir = Path(config.root_dir),
            custom_train_model_dir = Path(config.custom_train_model_dir),
            base_model_path = Path(config.base_model_path),
            custom_trained_model_path = Path(config.custom_trained_model_path),
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,
            params_include_top = self.params.INCLUDE_TOP,
            params_weights = self.params.WEIGHTS,
            params_classes = self.params.CLASSES
        )
        
        return base_model_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from pathlib import Path
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.efficientnet import preprocess_input


In [7]:
class BaseModel:
    def __init__(self, config: BaseModelConfig):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.params_image_size,
            weights = self.config.params_weights,
            include_top = self.config.params_include_top
        )
        
        self.save_model(path=self.config.base_model_path, model=self.model)
        
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
               
        flatten_in = Flatten()(model.output)
        
        prediction = Dense(
            units = classes,
            activation = "softmax"
        )(flatten_in)
        
        full_model = Model(
            inputs = model.input,
            outputs = prediction
        )
        
        optimizer = SGD(learning_rate=learning_rate)
        loss_fn = tf.keras.losses.CategoricalCrossentropy()
        full_model.compile(
            optimizer=optimizer,
            loss=loss_fn,
            metrics=["accuracy"]
        )
        
        full_model.summary()
        return full_model
        
    
    def custom_train_model(self):
        if not hasattr(self, 'model'):
            self.get_base_model()
        
        self.full_model = self._prepare_full_model(
            model = self.model,
            classes = self.config.params_classes,
            freeze_all = True,
            freeze_till = None,
            learning_rate = self.config.params_learning_rate
        )
        
        self.save_model(path=self.config.custom_trained_model_path, model=self.full_model)
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
        
    """def load_and_compile_model(self, path: Path):
        loaded_model = tf.keras.models.load_model(path)
        
        # Recreate the optimizer and compile the model
        optimizer = SGD(learning_rate=self.config.params_learning_rate)
        loss_fn = tf.keras.losses.CategoricalCrossentropy()
        loaded_model.compile(
            optimizer=optimizer,
            loss=loss_fn,
            metrics=["accuracy"]
        )
        
        return loaded_model"""
        
    

In [8]:
try:
    config = ModelConfigurationManager()
    base_model_config = config.get_base_model_config()
    base_model = BaseModel(config=base_model_config)
    base_model.get_base_model()
    base_model.custom_train_model()
    #base_model.load_and_compile_model(path="artifacts/custom_train_model/custom_trained_base_model.h5")
    #base_model.custom_train_model()
except Exception as e:
    raise e

[2024-06-02 00:59:21,818: INFO: common: yaml file: config\config.yaml loaded successfully!]
[2024-06-02 00:59:21,822: INFO: common: yaml file: params.yaml loaded successfully!]
[2024-06-02 00:59:21,824: INFO: common: created directory at: artifacts]
[2024-06-02 00:59:21,826: INFO: common: created directory at: artifacts/base_model]
[2024-06-02 00:59:21,828: INFO: common: created directory at: artifacts/custom_train_model]
[2024-06-02 00:59:23,997: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 528, 528, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 528, 528, 64)      1792      
                            